In [6]:
import os
import requests
import pandas as pd
from datetime import datetime
import numpy as np
from typing import List, Dict, TypedDict, Optional, Any

# Read API key
with open('../fred_api_key.txt', 'r') as f:
    api_key = f.read().strip()

# Base URL for FRED API
base_url = "https://api.stlouisfed.org/fred/"

# Function to make API requests
def fred_request(endpoint: str, params: Optional[Dict[str, Any]] = None) -> Optional[Dict[str, Any]]:
    if params is None:
        params = {}
    
    # Add API key to parameters
    params['api_key'] = api_key
    params['file_type'] = 'json'
    
    # Make request
    url = base_url + endpoint
    response = requests.get(url, params=params)
    
    # Check for successful response
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None
        
def get_inflation_data(series_id: str = "CPIAUCSL") -> Optional[pd.DataFrame]:  # Consumer Price Index for All Urban Consumers
    endpoint = "series/observations"
    params = {
        "series_id": series_id,
        "observation_start": "2023-01-01",  
        "frequency": "m",  
        "units": "pc1"     # Percent change from a year ago
    }
    result = fred_request(endpoint, params)
    
    if result and 'observations' in result:
        print(f"Retrieved {len(result['observations'])} observations for {series_id}")
        
        # Convert to DataFrame
        df = pd.DataFrame(result['observations'])
        df = df[['date', 'value']]
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        
        return df
    else:
        print("Failed to retrieve inflation data")
        return None


# Run functions to explore category ID 33509
if __name__ == "__main__":
    inflation_df = get_inflation_data()
    print(inflation_df.head())

Retrieved 27 observations for CPIAUCSL
         date    value
0  2023-01-01  6.34030
1  2023-02-01  5.95765
2  2023-03-01  4.93135
3  2023-04-01  4.94695
4  2023-05-01  4.12531
